In [35]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import altair as alt

pd.set_option('display.max_columns', None)

In [36]:
usecols = ['market','player_name','datagolf_base_history_fit','draftkings','fanduel','betmgm']

In [37]:
# get american odds

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),2)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),2)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),2)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),2)

# mc_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=make_cut&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
# make_cut = round(pd.read_csv(mc_path, usecols=usecols),2)

american_odds = pd.concat([win,top5,top10,top20])
american_odds = american_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']].convert_dtypes()

In [38]:
# get decimal odds - make expected values

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),3)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),3)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),3)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),3)

dec_odds = pd.concat([win,top5,top10,top20])
dec_odds = dec_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']]

dec_odds['fanduel_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['fanduel']) - 1,2)
dec_odds['draftkings_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['draftkings']) - 1,2)
dec_odds['betmgm_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['betmgm']) - 1,2)

ev = dec_odds[['market','player_name','fanduel_ev','draftkings_ev','betmgm_ev']]

In [40]:
# merge

odds = pd.merge(american_odds,ev,how='left',on=['market','player_name']).round(2)
odds = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_ev','draftkings','draftkings_ev','betmgm','betmgm_ev']].dropna()
odds.rename(columns={'datagolf_base_history_fit':'real_odds'}, inplace=True)

def plus_prefix(a):
    if a > 0:
        b = f"+{a}"
    else:
        b = a
    return b

odds['real_odds'] = odds['real_odds'].apply(plus_prefix)
odds['fanduel'] = odds['fanduel'].apply(plus_prefix)
odds['draftkings'] = odds['draftkings'].apply(plus_prefix)
odds['betmgm'] = odds['betmgm'].apply(plus_prefix)

odds

,market,player_name,real_odds,fanduel,fanduel_ev,draftkings,draftkings_ev,betmgm,betmgm_ev
0,win,"Scheffler, Scottie",+584,+450,-0.20,+450,-0.20,+400,-0.27
1,win,"McIlroy, Rory",+1348,+1100,-0.17,+1100,-0.17,+1100,-0.17
2,win,"Schauffele, Xander",+1362,+1400,0.01,+1400,0.01,+1400,0.01
3,win,"Rahm, Jon",+1600,+1100,-0.29,+1100,-0.29,+1200,-0.23
4,win,"Matsuyama, Hideki",+2530,+2000,-0.21,+2000,-0.21,+1800,-0.28
...,...,...,...,...,...,...,...,...,...
351,top_20,"Weir, Mike",+27219,+3500,-0.86,+3500,-0.86,+3500,-0.86
352,top_20,"Olazabal, Jose Maria",+890622,+20000,-1.00,+20000,-1.00,+5000,-1.00
353,top_20,"Singh, Vijay",+24295,+5500,-0.78,+5500,-0.78,+2800,-0.88
354,top_20,"Stubbs, Jasper",+559256,+11000,-1.00,+11000,-1.00,+2000,-1.00


In [32]:
odds_dict = {
    'win':'Win',
    'top_5':'Top 5',
    'top_10':'Top 10',
    'top_20':'Top 20'
}
odds['market'] = odds['market'].map(odds_dict)

In [41]:
styled_odds = odds.style.background_gradient(cmap="bone", subset=['fanduel_ev','draftkings_ev','betmgm_ev']).format(precision=2)

,market,player_name,real_odds,fanduel,fanduel_ev,draftkings,draftkings_ev,betmgm,betmgm_ev
0,win,"Scheffler, Scottie",+584,+450,-0.20,+450,-0.20,+400,-0.27
1,win,"McIlroy, Rory",+1348,+1100,-0.17,+1100,-0.17,+1100,-0.17
2,win,"Schauffele, Xander",+1362,+1400,0.01,+1400,0.01,+1400,0.01
3,win,"Rahm, Jon",+1600,+1100,-0.29,+1100,-0.29,+1200,-0.23
4,win,"Matsuyama, Hideki",+2530,+2000,-0.21,+2000,-0.21,+1800,-0.28
5,win,"Spieth, Jordan",+2977,+2200,-0.26,+2500,-0.16,+2200,-0.26
6,win,"DeChambeau, Bryson",+3088,+3500,0.11,+4000,0.29,+3300,0.07
7,win,"Fitzpatrick, Matt",+3175,+4000,0.29,+4000,0.29,+3300,0.07
8,win,"Niemann, Joaquin",+3688,+2800,-0.24,+2800,-0.24,+2800,-0.24
9,win,"Aberg, Ludvig",+3840,+3300,-0.14,+3000,-0.22,+2800,-0.26


In [34]:
odds.style\
    .background_gradient(cmap="bone", subset=['fanduel_ev','draftkings_ev','betmgm_ev'])\
    .background_gradient(cmap="bone", subset=['draftkings_ev'])\
    .background_gradient(cmap="bone", subset=['betmgm_ev']).format(precision=2)

,market,player_name,odds,fanduel,fanduel_ev,draftkings,draftkings_ev,betmgm,betmgm_ev
0,Win,"Scheffler, Scottie",+584,+450,-0.20,+450,-0.20,+400,-0.27
1,Win,"McIlroy, Rory",+1348,+1100,-0.17,+1100,-0.17,+1100,-0.17
2,Win,"Schauffele, Xander",+1362,+1400,0.01,+1400,0.01,+1400,0.01
3,Win,"Rahm, Jon",+1600,+1100,-0.29,+1100,-0.29,+1200,-0.23
4,Win,"Matsuyama, Hideki",+2530,+2000,-0.21,+2000,-0.21,+1800,-0.28
5,Win,"Spieth, Jordan",+2977,+2200,-0.26,+2500,-0.16,+2200,-0.26
6,Win,"DeChambeau, Bryson",+3088,+3500,0.11,+4000,0.29,+3300,0.07
7,Win,"Fitzpatrick, Matt",+3175,+4000,0.29,+4000,0.29,+3300,0.07
8,Win,"Niemann, Joaquin",+3688,+2800,-0.24,+2800,-0.24,+2800,-0.24
9,Win,"Aberg, Ludvig",+3840,+3300,-0.14,+3000,-0.22,+2800,-0.26


In [20]:
def plus_prefix(a):
    if a > 0:
        b = f"+{a}"
    else:
        b = a
    return b

In [24]:
odds.odds.apply(plus_prefix)

0         +584
1        +1348
2        +1362
3        +1600
4        +2530
        ...   
351     +27219
352    +890622
353     +24295
354    +559256
355     +29697
Name: odds, Length: 345, dtype: object

In [ ]:
pd.melt(odds_master,id_vars=['player_name','market'],value_vars=['datagolf_baseline', 'fanduel', 'fanduel_val',
       'draftkings', 'draftkings_val', 'betmgm', 'betmgm_val', 'caesars',
       'caesars_val'])

In [ ]:
odds_master = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_val','draftkings','draftkings_val','betmgm','betmgm_val']].dropna()#,'caesars','caesars_val']]
odds_master.columns = ['Market','Player','Odds','FD Odds','FD EV','DK Odds','DK EV','BetMGM Odds','MGM EV']#,'Caesars','Caesars EV']


odds_dict = {
    'win':'Win',
    'top_5':'Top 5',
    'top_10':'Top 10',
    'top_20':'Top 20'
}
odds_master['Market'] = odds_master['Market'].map(odds_dict)